In [190]:
#| label: libraries
#| include: false
import pandas as pd 
import altair as alt
import numpy as np
import sqlite3
import datadotworld as ddw
import os

## Elevator pitch

_Before starting my project I had to get to know the database, since it contains multiple datasets, it was necessary to indentify the database order and content on the datasets._

In [191]:
# Read data

sqlite_file = 'lahmansbaseballdb.sqlite'
con = sqlite3.connect(sqlite_file)


In [192]:

# results = pd.read_sql_query( 
#     'SELECT * FROM allstarfull',
#     con)

#results

In [193]:
# results1 = pd.read_sql_query( 
#     'SELECT * FROM collegeplaying',
#     con)



In [194]:
# results2 = pd.read_sql_query( 
#     'SELECT * FROM salaries',
#     con)


In [195]:
# battingpost1 = pd.read_sql_query( 
#     'SELECT * FROM battingpost',
#     con)


In [196]:
# type of tables in database
pd.read_sql_query('SELECT name from sqlite_master where type="table"',con
)

,name
0,allstarfull
1,appearances
2,awardsmanagers
3,awardsplayers
4,awardssharemanagers
5,awardsshareplayers
6,batting
7,battingpost
8,collegeplaying
9,divisions


__Highlight the grand questions__

## GRAND QUESTION 1

__Write an SQL query to create a new dataframe about baseball players who attended BYU-Idaho. The new table should contain five columns: playerID, schoolID, salary, and the yearID/teamID associated with each salary. Order the table by salary (highest to lowest) and print out the table in your report.__



_Below it's included a table with the baseball players who attended BYU-I._


In [197]:
# SQL CODE

byui_base = pd.read_sql_query('''
SELECT sal.playerid, schoolid, salary, sal.yearid 
FROM salaries as sal 
    JOIN collegeplaying as cp
        ON sal.playerid = cp.playerid
WHERE schoolid = "idbyuid"
ORDER BY salary DESC
''', con)

byui_base.head(10)

,playerID,schoolID,salary,yearID
0,lindsma01,idbyuid,4000000.0,2014
1,lindsma01,idbyuid,4000000.0,2014
2,lindsma01,idbyuid,3600000.0,2012
3,lindsma01,idbyuid,3600000.0,2012
4,lindsma01,idbyuid,2800000.0,2011
5,lindsma01,idbyuid,2800000.0,2011
6,lindsma01,idbyuid,2300000.0,2013
7,lindsma01,idbyuid,2300000.0,2013
8,lindsma01,idbyuid,1625000.0,2010
9,lindsma01,idbyuid,1625000.0,2010


## GRAND QUESTION 2

__This three-part question requires you to calculate batting average (number of hits divided by the number of at-bats)__

_The three tables with the data required are found below._


_a. Write an SQL query that provides playerID, yearID, and batting average for players with at least 1 at bat that year. Sort the table from highest batting average to lowest, and then by playerid alphabetically. Show the top 5 results in your report._

In [198]:
# a

batting = pd.read_sql_query('''
SELECT playerID, yearID, H+0.0/AB as bat_av
FROM batting
WHERE bat_av > 1
ORDER BY bat_av DESC, playerID ASC
LIMIT 5
''', con)

batting


,playerID,yearID,bat_av
0,suzukic01,2004,262.0
1,sislege01,1920,257.0
2,odoulle01,1929,254.0
3,terrybi01,1930,254.0
4,simmoal01,1925,253.0


_b. Use the same query as above, but only include players with at least 10 at bats that year. Print the top 5 results._

In [199]:
#b
batting_10 = pd.read_sql_query('''
SELECT playerID, yearID, H+0.0/AB as bat_av
FROM batting
WHERE bat_av >= 10
ORDER BY bat_av DESC, playerID ASC
LIMIT 10
''', con)

batting_10

# /*SELECT playerID, yearID, H, AB, H/AB AS bat_av*/

,playerID,yearID,bat_av
0,suzukic01,2004,262.0
1,sislege01,1920,257.0
2,odoulle01,1929,254.0
3,terrybi01,1930,254.0
4,simmoal01,1925,253.0
5,hornsro01,1922,250.0
6,kleinch01,1930,250.0
7,cobbty01,1911,248.0
8,sislege01,1922,246.0
9,suzukic01,2001,242.0


_c.Now calculate the batting average for players over their entire careers (all years combined). Only include players with at least 100 at bats, and print the top 5 results._

In [200]:
#c
batting_100 = pd.read_sql_query('''
SELECT playerID, yearID, H+0.0/AB as bat_av
FROM batting
WHERE bat_av >= 100
GROUP BY playerID
ORDER BY bat_av DESC, playerID ASC
LIMIT 5
''', con)

batting_100

,playerID,yearID,bat_av
0,suzukic01,2001,242.0
1,jacksjo01,1911,233.0
2,wanerll01,1927,223.0
3,williji01,1899,220.0
4,garrra01,1971,219.0


## GRAND QUESTION 3

__Pick any two baseball teams and compare them using a metric of your choice (average salary, home runs, number of wins, etc). Write an SQL query to get the data you need, then make a graph in Altair to visualize the comparison. What do you learn?__

_For this analysis I compared two teams for 20 years. Milwaukee Brewers as MIL and Minnesota Twins as MIN. Had to filter by years because before 1998 MIL did not exist. The interesting result of this plot is that both teams have a very similar number of wins per year, it's hard to tell which one is better since both have been really close and the team with more wins can change from one year to another.
_


In [201]:
teams = pd.read_sql_query( 
    'SELECT * FROM teams',
    con)


_include figures in chunks and discuss your findings in the figure._


In [202]:
#| label: GQ3 chart
#| code-summary: plot example
#| fig-cap: My useless chart
#| fig-align: center
# Include and execute your code here
graph = pd.read_sql_query('''
SELECT teamid, yearid, W
FROM teams
WHERE (teams.teamid = "MIL" OR teams.teamid = "MIN") AND yearid > 1998
''', con)

graph1= graph.__dataframe__


In [203]:
dft1= graph.groupby('teamID').agg(total_wins = ('W', sum)
).reset_index()



In [204]:
win_chart = alt.Chart(graph).mark_line(
    size=2
).encode(
    alt.X('yearID'),
    alt.Y('W'), alt.Color('teamID', legend= alt.Legend(title="Team"))
).properties(width=1000)

win_chart

alt.hconcat(
    win_chart.encode(x='yearID:O').properties(title='year=ordinal')

)

c:\Program Files\Python310\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [205]:
#graph.dtypes

## APPENDIX A (Additional Python Code)

```python
#paste other your code from your python file (.py) here
```